In [1]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType,ArrayType,LongType
import logging
import sys
import traceback
import ast
import json
import os
import sys
import traceback
import logging
from pyspark.sql.functions import col, expr,when,to_date ,udf, concat_ws,posexplode, from_json
from pyspark.sql.types import StructType, StructField, BooleanType, StringType, IntegerType, DateType, FloatType,DoubleType
from pyspark.sql import functions as F

In [3]:
# spark = SparkSession.builder \
#     .appName("MinIO with Delta Lake") \
#     .config("spark.driver.memory", "6g") \
#     .config("spark.executor.memory", "6g") \
#     .config("spark.driver.cores", "4") \
#     .config("spark.executor.cores", "4") \
#     .config("spark.dynamicAllocation.enabled", "true") \
#     .config("spark.dynamicAllocation.minExecutors", "1") \
#     .config("spark.dynamicAllocation.maxExecutors", "1") \
#     .config("spark.dynamicAllocation.initialExecutors", "1") \
#     .config("spark.sql.shuffle.partitions", "12") \
#     .config("spark.memory.offHeap.enabled", "true") \
#     .config("spark.memory.offHeap.size", "1g") \
#     .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
#     .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
#     .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
#     .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
#     .config("spark.hadoop.fs.s3a.path.style.access", "true") \
#     .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
#     .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
#     .config("delta.enable-non-concurrent-writes", "true") \
#     .config('spark.sql.warehouse.dir', "s3a://lakehouse/") \
#     .getOrCreate()


spark = SparkSession.builder \
    .appName("HybridRecommenderMaxPerformance") \
    .master("spark://spark-master:7077") \
    .config("spark.executor.memory", "5g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.cores", "4") \
    .config("spark.executor.cores", "4") \
    .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:5.1.3") \
    .config("spark.executor.instances", "1") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "4g") \
    .config("spark.jars", "jars/hadoop-aws-3.3.4.jar,jars/spark-sql-kafka-0-10_2.12-3.2.1.jar,jars/aws-java-sdk-bundle-1.12.262.jar,jars/delta-core_2.12-2.2.0.jar,jars/delta-storage-2.2.0.jar") \
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000") \
    .config("spark.hadoop.fs.s3a.access.key", "conbo123") \
    .config("spark.hadoop.fs.s3a.secret.key", "123conbo") \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore") \
    .config("delta.enable-non-concurrent-writes", "true") \
    .config("spark.sql.warehouse.dir", "s3a://lakehouse/") \
    .getOrCreate()

25/06/15 12:16:23 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
from pyspark.sql import functions as F
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import monotonically_increasing_id
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator



ratings = spark.read.format("delta").load("s3a://lakehouse/silver/ratings")
ratings = ratings.select("userId", "movieId", "rating")
df = ratings.withColumn('userId', ratings['userId'].cast('int')).\
withColumn('movieId', ratings['movieId'].cast('int')).withColumn('rating', ratings['rating'].cast('float'))
df.printSchema()

25/06/15 12:17:06 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)



In [5]:
ratings = ratings.select("userId", "movieId", "rating")
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: float (nullable = true)



In [6]:
df = ratings.withColumn('userId', ratings['userId'].cast('int')).\
withColumn('movieId', ratings['movieId'].cast('int')).withColumn('rating', ratings['rating'].cast('float'))
df.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: float (nullable = true)



In [9]:
train, validation, test = df.randomSplit([0.7, 0.15, 0.15], seed=0)
print("The number of ratings in each set: {}, {}, {}".format(train.count(), validation.count(), test.count()))

[Stage 27:==================================================>       (7 + 1) / 8]

The number of ratings in each set: 22399900, 4798379, 4797199


In [13]:
from pyspark.sql.functions import col, sqrt
def RMSE(predictions):
    squared_diff = predictions.withColumn("squared_diff", pow(col("rating") - col("prediction"), 2))
    mse = squared_diff.selectExpr("mean(squared_diff) as mse").first().mse
    return mse ** 0.5

In [14]:
# implement the model using ALS algorithm and find the right hyperparameters using Grid Search
from pyspark.ml.recommendation import ALS
import time  

def GridSearch(train, valid, num_iterations, reg_param, n_factors):
    min_rmse = float('inf')
    best_n = -1
    best_reg = 0
    best_model = None
    # run Grid Search for all the parameter defined in the range in a loop
    for n in n_factors:
        for reg in reg_param:
            als = ALS(rank = n, 
                      maxIter = num_iterations, 
                      seed = 0, 
                      regParam = reg,
                      userCol="userId", 
                      itemCol="movieId", 
                      ratingCol="rating", 
                      coldStartStrategy="drop")            
            model = als.fit(train)
            predictions = model.transform(valid)
            rmse = RMSE(predictions)     
            print('{} latent factors and regularization = {}: validation RMSE is {}'.format(n, reg, rmse))
            # track the best model using RMSE
            if rmse < min_rmse:
                min_rmse = rmse
                best_n = n
                best_reg = reg
                best_model = model
                
    pred = best_model.transform(train)
    train_rmse = RMSE(pred)
    # best model and its metrics
    print('\nThe best model has {} latent factors and regularization = {}:'.format(best_n, best_reg))
    print('traning RMSE is {}; validation RMSE is {}'.format(train_rmse, min_rmse))
    return best_model

In [15]:
# build the model using different ranges for Grid Search
from pyspark.sql.functions import col, sqrt
num_iterations = 10
ranks = [6, 8, 10, 12]
reg_params = [0.01, 0.03, 0.05, 0.07, 0.1]

start_time = time.time()
final_model = GridSearch(train, validation, num_iterations, reg_params, ranks)
print('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

6 latent factors and regularization = 0.01: validation RMSE is 0.8103752215843026


6 latent factors and regularization = 0.03: validation RMSE is 0.8038924409155549


6 latent factors and regularization = 0.05: validation RMSE is 0.8001335835076074


6 latent factors and regularization = 0.07: validation RMSE is 0.8009723908048109


6 latent factors and regularization = 0.1: validation RMSE is 0.8078194351853303


...

12 latent factors and regularization = 0.03: validation RMSE is 0.7889014973160202


12 latent factors and regularization = 0.05: validation RMSE is 0.7855243448734479


12 latent factors and regularization = 0.07: validation RMSE is 0.7883851964414105


12 latent factors and regularization = 0.1: validation RMSE is 0.8007412610186813



The best model has 12 latent factors and regularization = 0.05:
traning RMSE is 0.7167061052331951; validation RMSE is 0.7855243448734479
Total Runtime: 1312.56 seconds


In [16]:
final_model.save("s3a://lakehouse/data/CF_als_best_model")


In [5]:
from pyspark.ml.recommendation import ALSModel

temp = ALSModel.load("s3a://lakehouse/data/CF_als_best_model")

In [17]:
pred_test = final_model.transform(test)
print('The testing RMSE is ' + str(RMSE(pred_test)))

[Stage 3504:==========================================>             (6 + 2) / 8]

The testing RMSE is 0.7849833376593055


In [18]:
single_user = test.filter(test['userId']==12).select(['movieId','userId'])
single_user.show()

[Stage 3536:=====================================>                  (2 + 1) / 3]

+-------+------+
|movieId|userId|
+-------+------+
|    157|    12|
|    639|    12|
|   2640|    12|
|   8849|    12|
|   9340|    12|
|   9715|    12|
|  11224|    12|
+-------+------+



In [6]:
# Đọc dữ liệu movie_names và ratings_data từ Delta
movies  = spark.read.format("delta").load("s3a://lakehouse/gold/dim_movie")
movies  = movies .select("id", "title")
movies  = movies .withColumnRenamed("id", "movieId")
df_merge = spark.read.format("delta").load("s3a://lakehouse/gold/MergeDataTest")
from pyspark.ml.recommendation import ALSModel
df_list = spark.read.format("delta").load("s3a://lakehouse/data/bertTestHiHi")

temp = ALSModel.load("s3a://lakehouse/data/CF_als_best_model")

In [20]:
single_user.join(movies, single_user.movieId == movies.movieId, 'inner').show()


[Stage 3550:=====================================>                  (2 + 1) / 3]

+-------+------+-------+--------------------+
|movieId|userId|movieId|               title|
+-------+------+-------+--------------------+
|    157|    12|    157|Star Trek III: Th...|
|    639|    12|    639|When Harry Met Sa...|
|   2640|    12|   2640|            Heathers|
|   8849|    12|   8849|               Alfie|
|   9340|    12|   9340|         The Goonies|
|   9715|    12|   9715|         Hope Floats|
|  11224|    12|  11224|          Cinderella|
+-------+------+-------+--------------------+



In [21]:
reccomendations = final_model.transform(single_user)
reccomendations.orderBy('prediction',ascending=False).show()

[Stage 3553:=================================================>      (7 + 1) / 8]

+-------+------+----------+
|movieId|userId|prediction|
+-------+------+----------+
|   9715|    12| 3.2391589|
|  11224|    12| 3.1785722|
|   9340|    12| 3.1723292|
|    639|    12| 2.7267034|
|   2640|    12| 2.4407022|
|   8849|    12| 2.2240846|
|    157|    12| 0.8963401|
+-------+------+----------+



In [22]:
reccomendations.join(movies, reccomendations.movieId == movies.movieId, 'inner').show()


[Stage 3634:=================================================>      (7 + 1) / 8]

+-------+------+----------+-------+--------------------+
|movieId|userId|prediction|movieId|               title|
+-------+------+----------+-------+--------------------+
|    157|    12| 0.8963401|    157|Star Trek III: Th...|
|    639|    12| 2.7267034|    639|When Harry Met Sa...|
|   2640|    12| 2.4407022|   2640|            Heathers|
|   8849|    12| 2.2240846|   8849|               Alfie|
|   9340|    12| 3.1723292|   9340|         The Goonies|
|   9715|    12| 3.2391589|   9715|         Hope Floats|
|  11224|    12| 3.1785722|  11224|          Cinderella|
+-------+------+----------+-------+--------------------+



In [23]:
from pyspark.sql.functions import col, lit

# select a single user from the test set
user_id = 12
single_user_ratings = test.filter(test['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# display the movies the user has liked
print("Movies liked by user with ID", user_id)
single_user_ratings.join(movies, 'movieId').select('movieId', 'title', 'rating').show()

# generate recommendations for the user
all_movies = df.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# predict ratings for movies the user has not rated yet
recommendations = final_model.transform(movies_to_recommend.withColumn('userId', lit(user_id)))

# filter out the movies that the user has already rated or seen (this filters out the movies that the user has not liked as well)
recommendations = recommendations.filter(col('prediction') > 0)

# display the recommendations with movie names
print("Recommended movies for user with ID", user_id)
recommended_movies = recommendations.join(movies, 'movieId').select('movieId', 'title', 'prediction')

# Sort recommended movies by prediction in descending order
ordered_recommendations = recommended_movies.orderBy(col('prediction').desc())

# Display the ordered recommendations
ordered_recommendations.show()

Movies liked by user with ID 12


+-------+--------------------+------+
|movieId|               title|rating|
+-------+--------------------+------+
|    157|Star Trek III: Th...|   1.0|
|    639|When Harry Met Sa...|   3.5|
|   2640|            Heathers|   3.0|
|   8849|               Alfie|   4.0|
|   9340|         The Goonies|   3.0|
|   9715|         Hope Floats|   4.5|
|  11224|          Cinderella|   3.0|
+-------+--------------------+------+

Recommended movies for user with ID 12


+-------+--------------------+----------+
|movieId|               title|prediction|
+-------+--------------------+----------+
| 397403|   Brexit: The Movie|  6.505104|
|   1731|               Helen|  6.163009|
| 393841|      Pelli Choopulu|  5.947569|
| 412758|          Eat Locals| 5.9273725|
| 355177|Tig Notaro: Boyis...| 5.9270086|
| 261810|            Silenced| 5.8784776|
| 409502|     I'm Not Ashamed|  5.839202|
| 228198|Eu Não Faço a Men...|  5.811181|
| 407036|        On the Brain| 5.6171036|
| 461088|50 Kilos of Sour ...| 5.5664444|
|  65488|     Raintree County|  5.538809|
|  56759| Notre Dame de Paris|   5.46525|
|  11496|They Call Me Rene...| 5.4383755|
| 398854|A Midsummer Night...|  5.412646|
| 176088|   The Story of Luke|  5.370575|
|  63039|          Noah's Ark| 5.3683953|
|  42427|           Fair Play|  5.366928|
|  17022|Michael Jackson: ...| 5.3630924|
|  15199|Rent: Filmed Live...|  5.343305|
| 432025|The Art of Loving...|  5.272885|
+-------+--------------------+----

In [12]:
from pyspark.sql.functions import col, lit

# select a single user from the test set
user_id = 123
single_user_ratings = ratings.filter(ratings['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# display the movies the user has liked
print("Movies liked by user with ID", user_id)
single_user_ratings.join(df_merge, single_user_ratings['movieId'] == df_merge['id']).select("title", "genres", "director_names", "cast_names", "popularity", "vote_average", "vote_count", "rating", "movieId").show()

# generate recommendations for the user
all_movies = df.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# predict ratings for movies the user has not rated yet
recommendations = temp.transform(movies_to_recommend.withColumn('userId', lit(user_id)))

# filter out the movies that the user has already rated or seen (this filters out the movies that the user has not liked as well)
recommendations = recommendations.filter(col('prediction') > 4)

# display the recommendations with movie names
print("Recommended movies for user with ID", user_id)
recommended_movies = recommendations.join(df_merge, recommendations['movieId'] == df_merge['id']).select("title", "genres", "director_names", "cast_names", "popularity", "vote_average", "vote_count", "prediction", "movieId")

# Sort recommended movies by prediction in descending order
ordered_recommendations = recommended_movies.orderBy(col('popularity').desc())

# Display the ordered recommendations
ordered_recommendations.show()

Movies liked by user with ID 123
+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+------+-------+
|               title|              genres|     director_names|          cast_names|popularity|vote_average|vote_count|rating|movieId|
+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+------+-------+
|           Star Wars|[Action, Adventur...|     [George Lucas]|[Carrie Fisher, H...| 42.149697|         8.1|    6778.0|   5.0|     11|
|The Shawshank Red...|      [Crime, Drama]|   [Frank Darabont]|[Bob Gunton, Clan...| 51.645403|         8.5|    8358.0|   3.0|    278|
|    Schindler's List|[Drama, History, ...| [Steven Spielberg]|[Ben Kingsley, Ca...| 41.725123|         8.3|    4436.0|   4.5|    424|
|            Die Hard|  [Action, Thriller]|   [John McTiernan]|[Alan Rickman, Al...| 16.640522|         7.5|    4005.0|   4.0|    562|
|The Empire Strike...|

[Stage 191:=================================================>       (7 + 1) / 8]

+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+----------+-------+
|               title|              genres|     director_names|          cast_names|popularity|vote_average|vote_count|prediction|movieId|
+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+----------+-------+
|               Live!|          [Thriller]|   [Bill Guttentag]|[David Krumholtz,...| 12.376901|         5.7|      48.0| 4.5028553|  25218|
|Star Trek II: The...|[Action, Adventur...|   [Nicholas Meyer]|[DeForest Kelley,...|  7.160236|         7.3|     688.0| 4.2702622|    154|
|       He's a Dragon|[Adventure, Fanta...|[Indar Dzhendubaev]|[Maria Poezhaeva,...|  5.978316|         6.6|      38.0| 4.0405807| 370964|
|     Son of Paleface|[Comedy, Romance,...|    [Frank Tashlin]|[Bob Hope, Jane R...|  4.423289|         6.2|      13.0|   4.02585|  30801|
|Those Daring Youn...| [Adv

In [28]:
import numpy as np

In [18]:
from pyspark.sql.functions import col, lit
from pyspark.sql.types import FloatType
from pyspark.sql import functions as F
import numpy as np

# 1. Chọn 1 user
user_id = 123
single_user_ratings = ratings.filter(ratings['userId'] == user_id).select(['movieId', 'userId', 'rating'])

# 2. In các phim user đã rating
print("Phim user đã xem:")
single_user_ratings.join(df_merge, single_user_ratings['movieId'] == df_merge['id']).select(
    "title", "genres", "director_names", "cast_names", "popularity", "vote_average", "vote_count", "rating").show()

# 3. Lấy danh sách phim chưa xem
all_movies = df.select('movieId').distinct()
user_movies = single_user_ratings.select('movieId').distinct()
movies_to_recommend = all_movies.subtract(user_movies)

# 4. Dự đoán bằng CF
recommendations = temp.transform(movies_to_recommend.withColumn('userId', lit(user_id)))
recommendations = recommendations.filter(col('prediction') > 3.7)

# 5. Join info phim
recommended_movies = recommendations.join(df_merge, recommendations['movieId'] == df_merge['id']).select(
    "title", "genres",  "popularity", "vote_average", 
    "vote_count", "movieId", "prediction"
)

# 6. Lấy vector các phim user đã rating
rated_movies = single_user_ratings.join(df_list, single_user_ratings.movieId == df_list.id)

# 7. Tính vector trung bình
user_vec = rated_movies.select('vecs').rdd.map(lambda row: np.array(row['vecs'])).mean()
broadcast_user_vec = spark.sparkContext.broadcast(user_vec)

# 8. UDF tính cosine similarity
def cosine_similarity(v1, v2):
    v1 = np.array(v1)
    v2 = np.array(v2)
    numerator = float(np.dot(v1, v2))
    denominator = float(np.linalg.norm(v1) * np.linalg.norm(v2))
    return numerator / denominator if denominator != 0 else 0.0

cosine_udf = F.udf(lambda vec: float(cosine_similarity(broadcast_user_vec.value, vec)), FloatType())

# 9. Join vector vào các phim gợi ý và tính similarity
cb_filtered = recommended_movies.join(df_list, recommended_movies.movieId == df_list.id)
cb_filtered = cb_filtered.withColumn('similarity', cosine_udf(F.col('vecs')))

# 10. Sắp xếp theo similarity
cb_filtered = cb_filtered.orderBy(F.col('similarity').desc())

# 11. Hiển thị kết quả sau lọc bằng CB
print("Phim được lọc lại bằng CB (cosine):")
cb_filtered.select(
    "title", "genres", "popularity", "vote_average", 
    "vote_count", "similarity","prediction"
).show()


Phim user đã xem:
+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+------+
|               title|              genres|     director_names|          cast_names|popularity|vote_average|vote_count|rating|
+--------------------+--------------------+-------------------+--------------------+----------+------------+----------+------+
|           Star Wars|[Action, Adventur...|     [George Lucas]|[Carrie Fisher, H...| 42.149697|         8.1|    6778.0|   5.0|
|The Shawshank Red...|      [Crime, Drama]|   [Frank Darabont]|[Bob Gunton, Clan...| 51.645403|         8.5|    8358.0|   3.0|
|    Schindler's List|[Drama, History, ...| [Steven Spielberg]|[Ben Kingsley, Ca...| 41.725123|         8.3|    4436.0|   4.5|
|            Die Hard|  [Action, Thriller]|   [John McTiernan]|[Alan Rickman, Al...| 16.640522|         7.5|    4005.0|   4.0|
|The Empire Strike...|[Action, Adventur...|   [Irvin Kershner]|[Billy Dee Willia...| 19.47095

[Stage 438:=================================================>       (7 + 1) / 8]

+--------------------+--------------------+----------+------------+----------+----------+----------+
|               title|              genres|popularity|vote_average|vote_count|similarity|prediction|
+--------------------+--------------------+----------+------------+----------+----------+----------+
|      The Terminator|[Action, Science ...| 19.293562|         7.4|    4208.0|0.97331464| 3.8391197|
|Terminator 2: Jud...|[Action, Science ...| 22.661695|         7.7|    4274.0| 0.9712052| 3.9911008|
|Star Trek II: The...|[Action, Adventur...|  7.160236|         7.3|     688.0| 0.9703853| 4.2702622|
|2019: After the F...|[Action, Horror, ...|  1.323491|         4.5|      18.0| 0.9584577| 3.8921218|
|               Alien|[Action, Horror, ...|  23.37742|         7.9|    4564.0| 0.9579005| 3.7663982|
|      Steel Frontier|[Action, Science ...|  0.163373|         6.3|       3.0| 0.9574179|  4.374361|
|              Aliens|[Action, Horror, ...| 21.761179|         7.7|    3282.0|0.95667744| 3